# CS 345 Project
#### _Ben Spencer and Matteo Selvaggio_

#### **Part 0: Importing Data and Libraries**

In [19]:
# installing zstandard package to unpack .zst compressed files
# %pip install zstandard
import zstandard as zstd

# installing chess package which contains useful methods for processing chess boards.
# %pip install chess
import chess

# importing numpy and matplotlib
import numpy as np
import matplotlib.pyplot as plt

# importing sklearn methods
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# importing keras with GPU
# If using Anaconda, use the following commands in a local conda terminal:
# %pip install --upgrade pip
# %conda create -n tf tensorflow
# %conda activate tf
# %pip install tensorflow[and-cuda]
import tensorflow as tf

# import data
# the usecols parameter extracts the PuzzleID, FEN, and Game Moves columns only
# the skiprows parameter only receives the last 10,000 entries
X = np.loadtxt("lichess_db_puzzle.csv", delimiter=',', dtype=str, skiprows=3822157, usecols=(0,1))
y = np.loadtxt("lichess_db_puzzle.csv", delimiter=',', dtype=str, skiprows=3822157, usecols=(2))
print(f'Shape of the features array (X): {X.shape}')
print(f'Head of the features array: {X[:5]}\n')
print(f'Shape of the labels array (Y): {y.shape}')
print(f'Head of the labels array: {y[:10]}')


Shape of the features array (X): (10000, 2)
Head of the features array: [['zq34V' '2r3k1/5ppp/3Qb3/p3p3/4P3/5B2/PrP2PPP/2qRK2R b K - 2 24']
 ['zq35i'
  '4r1k1/2p1qpbp/1p4p1/p3p2n/4b3/1PP1PN1P/1P2BPPB/2QR2K1 w - - 2 21']
 ['zq3Ec' '8/8/3k4/2p2p1p/p1P4P/P3Pp2/4K3/8 w - - 0 47']
 ['zq3IO' 'r1b1r1k1/6pp/p1p5/3p4/2qB4/2P2QP1/P1P4P/5RK1 b - - 1 19']
 ['zq3JI' '8/3P2k1/5bp1/4N3/1p1K4/1P3R2/4r3/8 b - - 0 42']]

Shape of the labels array (Y): (10000,)
Head of the labels array: ['c1c2 d6d8 c8d8 d1d8' 'e2d3 e4d3 d1d3 e5e4 d3d2 e4f3'
 'e2f3 d6e5 f3f2 e5e4 f2e2 f5f4 e3f4 e4f4' 'c8h3 f3f7 g8h8 f7g7'
 'e2e5 f3f6 g7f6 d7d8q' 'f6f8 e4g2' 'd8d5 f5e6 d5e6 b3e6' 'e4d5 e1e8'
 'd6c5 g7c7 c5d5 c7c1' 'd1d3 e2e1 b1e1 e6e1']
